In [1]:
import json
import os
import logging
from typing import Dict, Any, List
from datetime import datetime
import random

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from crm_store import CRMStore
from skills.account_opening_tools import *

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Atomic functions testing

In [ ]:
# 0. Delete a client/prospect if already existing in cosmos
try:
    # Example of how to pull from environment variables:
    cosmosdb_endpoint = os.getenv("COSMOSDB_ENDPOINT") or ""
    crm_database_name = os.getenv("COSMOSDB_DATABASE_NAME") or ""
    crm_container_name = os.getenv("COSMOSDB_CONTAINER_CLIENT_NAME") or ""
    key=DefaultAzureCredential()
    
    crm_db = CRMStore(
            url=cosmosdb_endpoint,
            key=key,
            database_name=crm_database_name,
            container_name=crm_container_name
    )

    crm_db.delete_customer_profile("PROSP1400")

except Exception as e:
    logging.error(f"Error in delete_customer_profile: {str(e)}")


ERROR:root:Error in create_prospect: Invalid URL scheme or hostname: SplitResult(scheme='', netloc='', path='', query='', fragment='')


In [4]:

# 1. Create prospect
result = create_prospect("Tom", "Kocinsky", "1976/12/03", "US", "advisor")
print(result)

{"clientID": "PROSP3368", "firstName": "Tom", "lastName": "Kocinsky", "referral_source": "advisor", "id": "PROSP3368", "fullName": "Tom Kocinsky", "dateOfBirth": "1976/12/03", "nationality": "US", "contactDetails": {"email": "", "phone": ""}, "status": "KYC data collected successfully.", "pep_status": false, "risk_level": "", "risk_score": 0, "documents_provided": [], "name_screening_result": "None", "investmentProfile": {"riskProfile": "", "investmentObjectives": "", "investmentHorizon": ""}, "_rid": "TJJPANstunINAAAAAAAAAA==", "_self": "dbs/TJJPAA==/colls/TJJPANstunI=/docs/TJJPANstunINAAAAAAAAAA==/", "_etag": "\"030154fb-0000-4700-0000-67afa5000000\"", "_attachments": "attachments/", "_ts": 1739564288}


In [3]:
prospect_str = fetch_prospect_details("Albert Cook")
print(prospect_str)

{"clientID": "PROSP7960", "firstName": "Albert", "lastName": "Cook", "referral_source": "advisor", "id": "PROSP7960", "fullName": "Albert Cook", "dateOfBirth": "1983-11-27", "nationality": "Italy", "contactDetails": {"email": "", "phone": ""}, "status": "KYC data collected successfully.", "pep_status": false, "risk_level": "Low", "risk_score": 1, "documents_provided": [], "name_screening_result": "No match", "investmentProfile": {"riskProfile": "", "investmentObjectives": "", "investmentHorizon": ""}, "_rid": "TJJPANstunILAAAAAAAAAA==", "_self": "dbs/TJJPAA==/colls/TJJPANstunI=/docs/TJJPANstunILAAAAAAAAAA==/", "_etag": "\"4703a8f1-0000-4700-0000-67b5d48f0000\"", "_attachments": "attachments/", "_ts": 1739969679}


In [4]:
current_status = "KYC data collected successfully."
upd_prospect = json.loads(prospect_str)
upd_prospect['status'] = current_status
prospect = update_prospect_details(upd_prospect['clientID'],upd_prospect)

In [ ]:
prospect = json.loads(prospect_str)

prospect["documents_provided"] = ["passport", "proof_of_address"]
prospect["declared_source_of_wealth"] = "Income from employment"

"""
    Orchestrates the entire Prospect Information Collection process (2.1 to 2.6).
    Tracks sub-results in a single 'results' dictionary.
    """
results = {
    "initial_data": prospect,
    "kyc_collected": None,
    "sow_collected": None,
    "data_management_results": None,
    "name_screening_results": None,
    "client_profile": None,
    "compliance_risk_assessment": None
}

# 2.1 KYC Information Collection
results["kyc_collected"] = collect_kyc_info(prospect)

# 2.2 Source of Wealth Collection
results["sow_collected"] = collect_sow_info(prospect)

# 2.3 Data Management & AI Extraction
results["data_management_results"] = perform_data_management_ai_extraction(prospect)

# 2.4 Name Screening & Exception Handling
results["name_screening_results"] = perform_name_screening(prospect)

# 2.5 Client Profile Development
results["client_profile"] = create_client_profile(
    prospect, 
    results["name_screening_results"]
)

# 2.6 Compliance & Risk Assessment
results["compliance_risk_assessment"] = perform_compliance_risk_assessment(
    prospect
)
    
print(results)

{'initial_data': {'clientID': 'PROSP1400', 'firstName': 'Albert', 'lastName': 'Cook', 'referral_source': 'advisor', 'id': 'PROSP1400', 'fullName': 'Albert Cook', 'dateOfBirth': '1983/11/27', 'nationality': 'Italy', 'contactDetails': {'email': '', 'phone': ''}, 'status': 'First KYC checks passed', 'pep_status': False, 'risk_level': 'Low', 'risk_score': 1, 'documents_provided': ['passport', 'utility_bill'], 'name_screening_result': 'No match', 'investmentProfile': {'riskProfile': '', 'investmentObjectives': '', 'investmentHorizon': ''}, '_rid': 'TJJPANstunIKAAAAAAAAAA==', '_self': 'dbs/TJJPAA==/colls/TJJPANstunI=/docs/TJJPANstunIKAAAAAAAAAA==/', '_etag': '"14001efc-0000-4700-0000-67abbbcd0000"', '_attachments': 'attachments/', '_ts': 1739307981, 'declared_source_of_wealth': 'Income from employment'}, 'kyc_collected': {'status': 'KYC data collected successfully', 'collected_data': {'firstName': 'Albert', 'lastName': 'Cook', 'dateOfBirth': '1983/11/27', 'nationality': 'Italy'}}, 'sow_colle

In [ ]:
"""
Orchestrates steps 4.1 through 4.3 in the Document Collection phase.
"""
# Step 4.1: Gather official documents
document_info = check_required_documents(prospect)

# Step 4.2: Compliance & Verification Reporting
compliance_report = generate_compliance_verification_report(prospect)

print(document_info)
print(compliance_report)


## o1 orchestration 

In [5]:
from openai import AzureOpenAI

# Initialize OpenAI clients
def get_openai_client(key, endpoint, deployment):
    return AzureOpenAI(
        api_key=os.getenv(key),
        api_version="2024-02-15-preview",
        azure_endpoint=os.getenv(endpoint),
        azure_deployment=os.getenv(deployment)
    )

In [6]:
def call_o1(client, scenario):
    
        # Prompt templates
        O1_PROMPT = """
You are a an account opening assistant focusing on orchestrating a full end to end workflow of private banking investement account opening
that involve several steps.
You are a planner. The input you will receive (the scenario) include the current status of the workflow in the field "status".
Your task is to review the status and understand which are the next steps to execute next (following the example plan in prder below) by creating a plan from the next bullet point on.

You will have access to an LLM agent that is responsible for executing the plan that you create and will return results.

The LLM agent has access to the following functions:
{tools}

When creating a plan for the LLM to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - **Main actions are marked with letters** (e.g., A, B, C..).
    - **Sub-actions are under their relevant main actions** (e.g., A.1, B.2).
        - **Sub-actions should start on new lines**
    - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the status was approved, then...').
    - **For actions that require using one of the above functions defined**, write a step to call a function using backticks for the function name (e.g., `call the load_from_crm_by_client_fullname function`).
        - Ensure that the proper input arguments are given to the model for instruction. There should not be any ambiguity in the inputs.
    - **The last step** in the instructions should always be calling the `instructions_complete` function. This is necessary so we know the LLM has completed all of the instructions you have given it.
    - **Make the plan simple** Do not add steps on the plan when they are not needed.
    - **Generate summary** Before the `instructions_complete` ask the LLM to make a summary of the actions.
Use markdown format when generating the plan with each step and sub-step.

Please find the scenario below.
{scenario}

---

### Guidance Plan

Below is the ruleset of how you need to structure your final plan, including condition checks, steps/sub-steps, and function calls:

A. **Check if prospect exists in CRM**
   A.1 If the scenario provides 'fullName`, then:
       - `call the fetch_prospect_details function` with the required argument: `{"fullName": "..."}`.
       - If the result is `None` or indicates no match, respond back to the user asking to create a new prospect.
       - Else, go to step 2 directly.
   A.2 If the scenario does **not** provide a `fullName`, or we have no record, continue to step 2 to create a new prospect.

B. **Create a new prospect in the CRM** (if needed)
   B.1 `call the create_prospect function` with:
       ```json
       {
         "clientID" : "...",
         "first_name": "...",
         "last_name": "...",
         "dob": "YYYY-MM-DD",
         "nationality": "...",
         "referral_source": "..."
       }
       ```
   B.2 If you don't have from the scenario the necessary input to create a prospect, respond back by asking for them.
   B.3 If the function returns an `"error"` field, then stop and escalate (or log an error).  
       Else, proceed.

C. **Collect KYC Information** (Step 2.1)
   C.1 `call the collect_kyc_info function` with the `prospect_data` from CRM or newly created data:
    ```json
         {
            "name": "collect_kyc_info",
            "arguments": {
                "prospect_data": {
                    "clientID": ...,
                    "firstName": ...,
                    "lastName": ...,
                    "dateOfBirth": ...,
                    "nationality": ...
                }
            }
            }
       ```
   C.2 If the `"status"` returned is `"KYC incomplete"`, Stop the process (possibly prompt user to supply missing data).  
       Else continue to Step 2.2

D. **Gather Source of Wealth** (Step 2.2)
   D.1 `call the collect_sow_info function` with the same `prospect_data`.
   D.2 If `"source_of_wealth"` is `"Unknown"`, consider flagging for enhanced checks later. Proceed to Step 2.3

E. **Perform Data Management & AI Extraction** (Step 2.3)
   E.1 `call the perform_data_management_ai_extraction function` with `prospect_data`.
   Proceed to Step 2.4

F. **Perform Name Screening** (Step 2.4)
   F.1 `call the perform_name_screening function` with `prospect_data`.
   F.2 If the "name_screening_result" is `"Sanctions list match"`, consider flagging for enhanced checks later.
       Else continune with Step 2.5

G. **Create Client Profile** (Step 2.5)
   G.1 `call the create_client_profile function`, passing:
       ```json
       {
         "prospect_data": {...},
         "name_screening_result": {...}
       }
       ```
   G.2 If `"risk_level"` is `"High"`, note that in your plan for Enhanced Due Diligence (EDD) later.  
       Else proceed with Step 2.6

H. **Compliance & Risk Assessment** (Step 2.6)
   H.1 `call the perform_compliance_risk_assessment function` with:
       ```json
       {
         "prospect_data": {...}
       }
       ```
   H.2 If `"overall_status"` is `"High-risk client. Further Enhanced Due Diligence required."`, note that for additional steps.  
       Else continue to Step 3.1

I. **First Line of Defence** (Steps 3.1 & 3.2)
   I.1 `call the assign_first_line_of_defence function`, passing the updated `prospect_data`.
   I.2 After some waiting/polling or immediate result, `call the receive_first_line_of_defence function`.
       - If the returned `"status"` is `"First line of defence: approved"`, proceed.
       - Else Stop the process

J. **Check Required Documents** (Step 4.1)
    J.1 `call the check_required_documents function` with `prospect_data`.
    J.2 If the `"status"` is `"Documents missing"`, stop the process.  
         Else proceed with Step 4.2

K. **Generate Compliance Verification Report** (Step 4.2)
    K.1 `call the `generate_compliance_verification_repor` function` with `prospect_data`.
    Proceed to **Summary of Steps and Status** step

L. **Summary of Steps and Status**
    L.1 Summarize all the above outcomes and decisions:
         - KYC status
         - SOW check
         - Risk level
         - Internal due diligence findings
         - Final compliance report
         - Next steps (e.g., final account opening, or EDD if high risk)
    
M. **Complete Instructions**
    M.1 `call the instructions_complete function` to indicate all planned steps have finished.

**Conditions to highlight**:
- If any function returns an `"error"`, handle it logically (stop, escalate, or request more info).
- If a match is found on sanctions lists or `"risk_level"` is `"High"`, consider special escalation or Enhanced Due Diligence steps.

**End of Plan**

"""       
        
        prompt= O1_PROMPT.replace("{tools}",str(TOOLS)).replace("{scenario}",str(scenario))
     
        response = client.chat.completions.create(
            model=os.getenv("O1_MINI_OPENAI_DEPLOYMENT_NAME"),
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        plan = response.choices[0].message.content
        print(f"📟 Response from o1 plan: {plan}")
        return plan

### Testing o1 planner

In [7]:
o1_client = get_openai_client("O1_OPENAI_API_KEY", "O1_OPENAI_ENDPOINT", "O1_MINI_OPENAI_DEPLOYMENT_NAME")

prospect_test = fetch_prospect_details("Albert Cook")

#o1 planner agent part
o1_response = call_o1(o1_client, prospect_test)
o1_reply = {
    'role': 'assistant',
    'name': 'o1-mini Planner',
    'content': o1_response
}
print( o1_reply )

📟 Response from o1 plan: ```markdown
### Private Banking Investment Account Opening Plan

D. **Gather Source of Wealth** (Step 2.2)
   - **D.1** `call the collect_sow_info` function with:
     ```json
     {
       "prospect_data": {
         "clientID": "PROSP7960",
         "firstName": "Albert",
         "lastName": "Cook",
         "dateOfBirth": "1983-11-27",
         "nationality": "Italy"
       }
     }
     ```
   - **D.2** If `"source_of_wealth"` is `"Unknown"`, consider flagging for enhanced checks later. Proceed to Step E.

E. **Perform Data Management & AI Extraction** (Step 2.3)
   - **E.1** `call the perform_data_management_ai_extraction` function with:
     ```json
     {
       "prospect_data": {
         "clientID": "PROSP7960",
         "documents_provided": []
       }
     }
     ```
   - **E.2** Proceed to Step F.

F. **Perform Name Screening** (Step 2.4)
   - **F.1** `call the perform_name_screening` function with:
     ```json
     {
       "prospect_data": {
  

Execution (4o-mini) agent

In [8]:
def call_gpt4o(client, plan):
        GPT4O_SYSTEM_PROMPT = """
You are a helpful assistant responsible for executing a plan about account opening.
Your task is to:
1. Follow the plan exactly as written
2. Use the available tools to execute each step
3. Provide clear explanations of what you're doing
4. Always respond with some content explaining your actions
5. Call the instructions_complete function only when all steps are done
6. Never write or execute code
7. In your response, do not add things like "I have succesfully do this and that..." or "This should provide you with the content you asked for..."

PLAN TO EXECUTE:
{plan}

Remember to explain each action you take and provide status updates.
"""
        
        gpt4o_policy_prompt = GPT4O_SYSTEM_PROMPT.replace("{plan}", plan)
        messages = [{'role': 'system', 'content': gpt4o_policy_prompt}]

        while True:
            response = client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                messages=messages,
                tools=TOOLS,
                parallel_tool_calls=False
            )
            #self.logger.info(f" Response from 4o agent:\n {response}")
            
            assistant_message = response.choices[0].message.model_dump()
            messages.append(assistant_message)
   
            if not response.choices[0].message.tool_calls:
                continue

            for tool in response.choices[0].message.tool_calls:
                if tool.function.name == 'instructions_complete':
                    return messages

                function_name = tool.function.name 
              
                print(f"📟 Executing function: {function_name}")
                try:
                    arguments = json.loads(tool.function.arguments)
                    print(f"📟 ...with arguments: {arguments}")
                    function_response = FUNCTION_MAPPING[function_name](**arguments)
                  
                    print( f"{function_name}: {json.dumps(function_response)}")
                    print("Function executed successfully!")
                    #print(function_response)
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "content": json.dumps(function_response)
                    })
                    
                except Exception as e:
                    print('error', f"Error in {function_name}: {str(e)}")

## Testing 4o Execution

In [ ]:
client = get_openai_client("AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME")

#4o executor part
ex_response = call_gpt4o(client, o1_response)

# Filter assistant messages that have actual content
assistants_4o_contents = [
    msg['content'] for msg in ex_response 
    if msg.get('role') == 'assistant' and msg.get('content') is not None
]

print( f"4o status: {assistants_4o_contents}")

📟 Executing function: collect_sow_info
📟 ...with arguments: {'prospect_data': {'clientID': 'PROSP7960', 'firstName': 'Albert', 'lastName': 'Cook', 'dateOfBirth': '1983-11-27', 'nationality': 'Italy'}}
collect_sow_info: {"source_of_wealth": "Employment", "status": "SOW information captured"}
Function executed successfully!
📟 Executing function: perform_data_management_ai_extraction
📟 ...with arguments: {'prospect_data': {'clientID': 'PROSP7960', 'documents_provided': []}}
perform_data_management_ai_extraction: {"extracted_fields": {"passport_number": null, "proof_of_address_verified": false}, "status": "Documents AI extraction completed"}
Function executed successfully!
📟 Executing function: perform_name_screening
📟 ...with arguments: {'prospect_data': {'clientID': 'PROSP7960', 'firstName': 'Albert', 'lastName': 'Cook', 'dateOfBirth': '1983-11-27', 'nationality': 'Italy'}}
perform_name_screening: {"name_screening_result": "No match", "status": "Cleared"}
Function executed successfully!
